# Midplane Section Analysis
Code to analyze the result of `midplane_section.py`

## Imports

In [1]:
# %% Imports
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.integrate
import scipy.interpolate

## Visualization
import matplotlib.pyplot as plt
import matplotlib as mpl

import pyvista as pv

from tqdm import tqdm
%matplotlib qt

## Utility libraries
from netCDF4 import Dataset
import glob

from c1lgkt.fields.equilibrium import Equilibrium
from c1lgkt.fields.field_handlers import GaussHermiteFieldHandler, XgcZonalFieldHandler, GaussHermiteFunction
from c1lgkt.fields.field_interpolators import compute_balloon_interpolation
from c1lgkt.fields.geometry_handlers import XgcGeomHandler

import c1lgkt.particles.particle_motion as particle_motion
import c1lgkt.particles.particle_tools as particle_tools

In [2]:
# %% Load files and perform setup

eq = Equilibrium.from_eqdfile(R'D:\Documents\IFS\hmode_jet\D3D141451.eqd')

xgcdata = Dataset(R'D:\Documents\Globus\XGC1.nc')

geom_files = {
    'ele_filename': R'D:\Documents\IFS\hmode_jet\Seo.eqd.ele',
    'fdmat_filename': R'D:\Documents\IFS\hmode_jet\fdmat.pkl',
    'min_e_filename': R'D:\Documents\IFS\hmode_jet\min_E_mat.pkl'
}
geom = XgcGeomHandler(eq, xgcdata, **geom_files)

uph = np.load('./outputs/phase_vel.npz')['u_lstsq']

# Set up zonal interpolation function
tind = 401
zpot = xgcdata['pot00'][tind,:]
zpot_psi = xgcdata['psi00'][:]
interp_zpot = scipy.interpolate.CubicSpline(zpot_psi, zpot, extrapolate=True)
zonalFields = XgcZonalFieldHandler(eq, xgcdata, 401)

# Set up ballooning mode interpolator
fit_results = np.load('./outputs/fit_results.npz', allow_pickle=True)
params_g, params_gh = fit_results['params_g'], fit_results['params_gh']

# Set up the interpolator
mode = GaussHermiteFunction(params_g[:4], params_gh)
interp_balloon = [(39, mode)]

ballFields = GaussHermiteFieldHandler(geom, interp_zpot, interp_balloon)

tind0 = 401 #424, 386

omega_frame = -uph[tind0,196]*geom.q_surf[196]*1e-3
rotating_frame = particle_motion.RotatingFrameInfo(0, omega_frame, tind0)
t0 = rotating_frame.t0

In [3]:
# %% Load data and compute Poincare sections

filelabel = 'midplane_deut_trapped_r50'
output_dir = 'D:/Documents/IFS/hmode_jet/outputs/'

paths = glob.glob(output_dir+'sections/{}/[0-9][0-9][0-9][0-9][0-9].npz'.format(filelabel))

num_saves = len(paths)
print('num_saves: {}'.format(num_saves))
ncheckpoint = 800

t = np.empty(num_saves*ncheckpoint)
y = None
dy = None

# TODO: Refactor so we don't load the entire trajectory into memory to compute the punctures
for kc in range(num_saves):
    with np.load(output_dir + 'sections/{}/{:05d}.npz'.format(filelabel, kc)) as data:
        t[kc*ncheckpoint:(kc+1)*ncheckpoint] = data['t']
        if kc == 0:
            y = np.empty((data['y'].shape[0], len(t)))
            dy = np.empty((data['y'].shape[0], len(t)))
            
        y[:,kc*ncheckpoint:(kc+1)*ncheckpoint] = data['y']
        dy[:,kc*ncheckpoint:(kc+1)*ncheckpoint] = data['dy']

nump = y.shape[0]//5

# Compute Poincare section
ppunc, npunc = particle_tools.compute_midplane_punctures(t, y, geom)
ppunc2, npunc2 = particle_tools.compute_toroidal_punctures(t, y, rotating_frame, period=(2*np.pi/39))


num_saves: 387


## Analysis

In [ ]:
# %% Plot trajectories in the (R,Z) plane atop the ballooning mode

plt.figure()

ax = plt.subplot(111)

eq.plot_magnetic_geometry(ax)
for k in [0]:
    ax.plot(y[k + 0*nump,:], y[k + 2*nump,:], marker='.')

rplot, zplot = np.linspace(2.15, 2.26, 256), np.linspace(-0.2, 0.2, 1024)
rgrid, zgrid = np.meshgrid(rplot, zplot)
varphigrid = np.zeros(rgrid.shape)

rflat, zflat, varphiflat = rgrid.flatten(), zgrid.flatten(), varphigrid.flatten()

psi_ev, ff_ev = eq.compute_psi_and_ff(rflat, zflat)

phi = compute_balloon_interpolation(
    0.0, rflat, zflat, varphiflat, psi_ev, eq, geom, [interp_balloon], gradient=False)

phigrid = phi.reshape(rgrid.shape)

plt.pcolormesh(rplot, zplot, phigrid, cmap='viridis')

plt.xlabel('R')
plt.ylabel('Z')

Text(0, 0.5, 'Z')

In [ ]:
# %% Plot trajectories in phi, theta plane

def centerize(z):
    return np.mod(z+np.pi,2*np.pi)-np.pi

plt.figure()

k = 1

theta = geom.compute_theta(y[k + 0*nump], y[k + 2*nump])
psi = eq.interp_psi.ev(y[k + 0*nump], y[k + 2*nump])
#plt.scatter(np.mod(y[k + 1*nump],2*np.pi), centerize(theta), c=psi, s=9*(72/100.0)**2)

plt.scatter(psi / eq.psix, theta, s=4*(72/100.0)**2, lw=0, c=t)

plt.xlabel('psi_n')
plt.ylabel('theta')

In [11]:
# %% Plot trajectories vs. time

k = 0

plt.figure()
plt.plot(t, dy[k + 0*nump], marker='.')
#plt.twinx()
plt.plot(t, dy[k + 2*nump], marker='.', c='tab:orange')

In [9]:
# %% Plot the outer midplane section

fig, axs = plt.subplots(2,1, sharex='all')

ravgs = np.empty(nump)
iota = np.empty(nump)
bounce_time = np.empty(nump)

rmax = np.max(np.concatenate([ppunc[k][1][0,:] for k in range(nump)]))
rmin = np.min(np.concatenate([ppunc[k][1][0,:] for k in range(nump)]))

for k in range(nump):
    ## Compute the rotation number
    varphi = np.unwrap(ppunc[k][1][1,:] - omega_frame * (ppunc[k][0] - t0))
    # Discrete time s
    s = np.linspace(0,1, len(varphi)-1, endpoint=False)
    # weight function
    w = np.zeros(len(varphi)-1)
    w[1:] = np.exp(-1.0 / (s[1:] * (1-s[1:])))
    iota[k] = np.sum(np.diff(varphi) * w) / (2*np.pi*np.sum(w))

    ## Compute the average r
    rmid = (ppunc[k][1][0,1:] + ppunc[k][1][0,:-1])/2.0
    ravgs[k] = np.sum(rmid * w) / np.sum(w)

    ## Compute the bounce time
    bounce_time[k] = np.sum(np.diff(ppunc[k][0]) * w) / np.sum(w)

    ntor = 39

    ## Compute color to plot based on the radial location
    norm = mpl.colors.Normalize(vmin=rmin, vmax=rmax)
    color = mpl.cm.plasma(norm(ravgs[k]))

    ## Plot the Poincare section, colored by trajectory
    axs[0].scatter(ppunc[k][1][0,:], np.mod(ppunc[k][1][1,:] - omega_frame * (ppunc[k][0] - t0),2*np.pi/ntor), s=4*(72/100.0)**2, lw=0, color=color)

    ## Plot the rotation number
    axs[1].scatter(ravgs[k], iota[k], color=color)


#plt.suptitle('Trapped electron Poincare section')

axs[0].set_ylabel(R'Toroidal angle $\varphi$')
#axs[0].set_ylabel(R'Major radius $R$')
axs[1].set_xlabel(R'Major radius $R$')
axs[1].set_ylabel(R'Rotation number $\iota$')

plt.tight_layout()

In [ ]:

# %% Plot the poloidal cross-section section in (R,Z)

fig, axs = plt.subplots(1,2,sharex='all',sharey='all')
#fig, axs = plt.subplots(1,2)

ravgs = np.empty(nump)

eq.plot_magnetic_geometry(axs[0])
eq.plot_magnetic_geometry(axs[1])

#axs[0].pcolormesh(rplot, zplot, phigrid, cmap='viridis')

for k in range(0,nump):
    #ravgs[k] = np.average(np.concatenate((ppunc[k][1][0,:], npunc[k][1][0,:])))

    #cplot = 'tab:blue' if ravgs[k] > 2.2259 else 'tab:orange'
    #cplot = 'tab:blue'

    axs[0].scatter(ppunc2[k][1][0,:], ppunc2[k][1][2,:], s=1*(72/100.0)**2, lw=0)
    axs[1].scatter(npunc2[k][1][0,:], npunc2[k][1][2,:], s=1*(72/100.0)**2, lw=0)

axs[0].set_xlabel(R'$R$ [m]')
axs[1].set_xlabel(R'$R$ [m]')
axs[0].set_ylabel(R'$Z$ [m]')
axs[1].set_ylabel(R'$Z$ [m]')

axs[0].set_title(R'$v_{\parallel} \hat{\mathbf{b}}\cdot\hat{\boldsymbol{\varphi}}$ > 0')
axs[1].set_title(R'$v_{\parallel} \hat{\mathbf{b}}\cdot\hat{\boldsymbol{\varphi}}$ < 0')

plt.tight_layout()

In [5]:
# %% Plot poloidal cross-section in (Z,v_||)

plt.figure()

fig, axs = plt.subplots(2,1)

rotation_number = np.empty(nump)

for k in range(0,nump):
    # Sort the combined punctures by their puncture time
    tpunc = np.concatenate((ppunc2[k][0], npunc2[k][0]))
    tsort = np.argsort(tpunc)
    tpunc = tpunc[tsort]
    print(np.average(np.diff(tpunc)))

    # Unpack and collect together the positive and negative punctures
    r = np.concatenate((ppunc2[k][1][0,:], npunc2[k][1][0,:]))[tsort]
    z = np.concatenate((ppunc2[k][1][2,:], npunc2[k][1][2,:]))[tsort]
    vll = np.concatenate((ppunc2[k][1][3,:], npunc2[k][1][3,:]))[tsort]

    # Compute theta
    psi = eq.interp_psi.ev(r, z)
    gtheta = np.arctan2(z-eq.zaxis, r-eq.raxis)
    dtheta = geom.interp_gdtheta_grid(psi, gtheta)
    theta = gtheta + dtheta

    ## Compute rotation numbers using weighted birkhoff average

    # Compute winding theta around the origin in (theta, vll) space
    winding = np.unwrap(np.arctan2(theta, vll))
    
    # discrete time s
    s = np.arange(len(theta)-1) / (len(theta)-1)
    # weight function
    w = np.zeros(len(theta)-1)
    w[1:] = np.exp(-1.0 / (s[1:] * (1-s[1:])))
    # WBA formula for rotation number
    rotation_number[k] = np.sum(w * np.diff(winding)) / np.sum(w)

    # Compute average area enclosed by the trajectory
    area = np.sum((np.diff(theta) * (vll[:-1] + vll[1:]) / 2.0) * w) / rotation_number[k]

    # Plot, colored by rotation number
    bounds = (0.07214376658924238, 0.08324086979945851)
    norm = mpl.colors.Normalize(vmin=bounds[0], vmax=bounds[1])
    color = mpl.cm.plasma(norm(rotation_number[k]))

    axs[0].scatter(theta, vll, s=4*(72/100.0)**2, lw=0, color=color)
    if np.abs(rotation_number[k]) > 0.01:
        axs[1].scatter(area, rotation_number[k], color=color)

print((np.min(rotation_number), np.max(rotation_number)))


0.005867482787194623
0.006088601543336555
0.0058785649915219
0.0060993560135515264
0.006012407714508888
0.0061330884573050195
0.005610253848820004
0.006246916912522602
0.005735148175148647
0.006383095721881616
0.00611952977029218
0.006406956380704322
0.006133933737228225
0.006265944806902099
0.005886577831411924
0.006132623411415823
(0.07206973881053441, 0.08323632719108245)


In [ ]:

# %% Poloidal cross-section flux coordinates (psi, theta)


fig, axs = plt.subplots(2,1,sharex='all')

ravgs = np.empty(nump)

maxrot = -np.inf
minrot = np.inf

for k in range(nump):
    #ravgs[k] = np.average(np.concatenate((ppunc[k][1][0,:], npunc[k][1][0,:])))

    
    #cplot = 'tab:blue'

    thetap = geom.compute_theta(ppunc2[k][1][0,:], ppunc2[k][1][2,:])
    psip = eq.interp_psi.ev(ppunc2[k][1][0,:], ppunc2[k][1][2,:])
    thetan = geom.compute_theta(npunc2[k][1][0,:], npunc2[k][1][2,:])
    psin = eq.interp_psi.ev(npunc2[k][1][0,:], npunc2[k][1][2,:])

    #axs[0].scatter(thetap, ppunc2[k][1][3,:], s=1*(72/100.0)**2, lw=0)
    #axs[1].scatter(thetan, npunc2[k][1][3,:], s=1*(72/100.0)**2, lw=0)

    #cplotp = 'tab:blue' if np.average(psip) > geom.psi_surf[200] else 'tab:orange'
    #cplotn = 'tab:blue' if np.average(psin) > geom.psi_surf[200] else 'tab:orange'

    # eq.interp_router(psip/eq.psix)

    if k%2 == 1:
        axs[0].scatter(thetap, psip, s=4*(72/100.0)**2, lw=0)
    else:
        axs[1].scatter(thetan, psin, s=4*(72/100.0)**2, lw=0)

axs[1].set_xlabel(R'Poloidal angle $\theta$')
axs[0].set_ylabel(R'Mapped outboard midplane $R$')

plt.suptitle(R'Passing ion Poincare section')

In [ ]:
# %% Check bounce times and poloidal transit times
plt.figure()
plt.scatter(ravgs, bounce_time, marker='.', c='tab:orange')
print(bounce_time)

[0.48498742 0.48449365 0.48475963 0.48446286 0.48467071 0.4841977
 0.48845671 0.48351919 0.48576335 0.48362654 0.48445724 0.48364858
 0.48418176 0.4833157  0.48499821 0.48390607]



In [ ]:
plt.close('all')

In [ ]:
t

In [ ]:
xgcdata['t'][:]*1e3

masked_array(data=[0.0039514, 0.0079028, 0.0118542, 0.0158056, 0.019757 ,
                   0.0237084, 0.0276598, 0.0316112, 0.0355626, 0.039514 ,
                   0.0434654, 0.0474168, 0.0513682, 0.0553196, 0.059271 ,
                   0.0632224, 0.0671738, 0.0711252, 0.0750766, 0.079028 ,
                   0.0829794, 0.0869308, 0.0908822, 0.0948336, 0.098785 ,
                   0.1027364, 0.1066878, 0.1106392, 0.1145906, 0.118542 ,
                   0.1224934, 0.1264448, 0.1303962, 0.1343476, 0.138299 ,
                   0.1422504, 0.1462018, 0.1501532, 0.1541046, 0.158056 ,
                   0.1620074, 0.1659588, 0.1699102, 0.1738616, 0.177813 ,
                   0.1817644, 0.1857158, 0.1896672, 0.1936186, 0.19757  ,
                   0.2015214, 0.2054728, 0.2094242, 0.2133756, 0.217327 ,
                   0.2212784, 0.2252298, 0.2291812, 0.2331326, 0.237084 ,
                   0.2410354, 0.2449868, 0.2489382, 0.2528896, 0.256841 ,
                   0.2607924, 0.264743

In [ ]:
plt.close('all')